In [0]:
from google.colab import files
uploaded = files.upload()

Saving simulation.zip to simulation.zip


In [0]:
!unzip simulation.zip

Archive:  simulation.zip
  inflating: CONFIG/POS_TAG_CONFIG.json  
  inflating: CONFIG/WORDS_CONFIG.json  
  inflating: model/model.h5          
  inflating: model/model.json        
  inflating: model/model_history.json  
  inflating: model/model_history_with_pos.json  
  inflating: model/model_with_pos.h5  
  inflating: model/model_with_pos.json  
   creating: module/__pycache__/
  inflating: module/__pycache__/EDA.cpython-36.pyc  
  inflating: module/__pycache__/preprocessing.cpython-36.pyc  
  inflating: module/__pycache__/test.cpython-36.pyc  
  inflating: module/EDA.py           
  inflating: module/preprocessing.py  
  inflating: module/test.py          


In [0]:
!pip install git+https://www.github.com/keras-team/keras-contrib.git
!pip uninstall keras
!pip install keras==2.2.4
import nltk
nltk.download('punkt')
nltk.download('averaged_perceptron_tagger')

  Cloning https://www.github.com/keras-team/keras-contrib.git to /tmp/pip-req-build-zm2xy3j6
  Running command git clone -q https://www.github.com/keras-team/keras-contrib.git /tmp/pip-req-build-zm2xy3j6
  Created wheel for keras-contrib: filename=keras_contrib-2.0.8-cp36-none-any.whl size=101065 sha256=1419157a5d22c0314bb2f02c6ed8afd59ba9373faff2ff643409a331acd022d1
  Stored in directory: /tmp/pip-ephem-wheel-cache-cemp4xx4/wheels/11/27/c8/4ed56de7b55f4f61244e2dc6ef3cdbaff2692527a2ce6502ba
Successfully built keras-contrib
Uninstalling Keras-2.2.5:
  Would remove:
    /usr/local/lib/python3.6/dist-packages/Keras-2.2.5.dist-info/*
    /usr/local/lib/python3.6/dist-packages/docs/*
    /usr/local/lib/python3.6/dist-packages/keras/*
  Would not remove (might be manually added):
    /usr/local/lib/python3.6/dist-packages/docs/md_autogen.py
    /usr/local/lib/python3.6/dist-packages/docs/update_docs.py
Proceed (y/n)? y
  Successfully uninstalled Keras-2.2.5
     |████████████████████████████

True

In [0]:
import os
import json
import numpy as np
from module.preprocessing import get_index_list_of_sentences
from keras.models import model_from_json
from keras.optimizers import Adam
from keras_contrib.layers import CRF
from module.test import get_arg_list
from nltk.tokenize import word_tokenize
from nltk.tag import pos_tag

import warnings
warnings.filterwarnings("ignore")

Using TensorFlow backend.


In [0]:
input_sentence = input('Input your sentence: ')

# config variable
CONFIG_ROOT = 'CONFIG'
CONFIG_NAME = 'POS_TAG_CONFIG.json'
CONFIG_WORDS = 'WORDS_CONFIG.json'

CONFIG_PATH = os.path.join(CONFIG_ROOT, CONFIG_NAME)
WORDS_PATH = os.path.join(CONFIG_ROOT, CONFIG_WORDS)

with open(CONFIG_PATH) as config:
    config_dic = json.load(config)

with open(WORDS_PATH) as words_path:
    words_config = json.load(words_path)

max_sentence_length = config_dic['max_sentence_length']
index_to_word_dic = words_config['index_to_word_dic_with_pos']
vocab_size = words_config['vocab_size_with_pos']
ner_to_index_dic = words_config['ner_to_index_dic']
B_PER_index = ner_to_index_dic['B-PER']
I_PER_index = ner_to_index_dic['I-PER']
n_labels = len(ner_to_index_dic)
word_to_index_dic = {tuple(value): int(key) for key, value in index_to_word_dic.items()}


def padding(x):
    if len(x) >= max_sentence_length:
        return x[:max_sentence_length]
    else:
        return x + [0] * (max_sentence_length - len(x))


input_sentence_list = pos_tag(word_tokenize(input_sentence))  # input_sentence.lower()
input_sentence_list = list(map(lambda x: (x[0], x[1]), input_sentence_list))
input_sentence_list = [(word, pos)for word, pos in input_sentence_list]

input_sentence_with_index = get_index_list_of_sentences(
    [[(word, "O") for word in input_sentence_list]], word_to_index_dic)
final_input = np.array([padding(x) for x in input_sentence_with_index],  dtype='f')

# 모델 불러오기
MODEL_ROOT = 'model'
MODEL_JSON = 'model_with_pos.json'
MODEL_H5 = 'model_with_pos.h5'


MODEL_JSON_PATH = os.path.join(MODEL_ROOT, MODEL_JSON)
MODEL_H5_PATH = os.path.join(MODEL_ROOT, MODEL_H5)

with open(MODEL_JSON_PATH, 'r') as f:
    loaded_model_json = f.read()

loaded_model = model_from_json(loaded_model_json, custom_objects={'CRF': CRF})
loaded_model.load_weights(MODEL_H5_PATH)

crf = CRF(n_labels)

loaded_model.compile(loss=crf.loss_function,
                     optimizer=Adam(0.001), metrics=[crf.accuracy])

# 예측하기

input_predict = loaded_model.predict(final_input)

len_input = len(input_sentence_list)
input_predict_word_index = get_arg_list(input_predict)[:len_input]


# 출력
division_length = 80
print("result  ".ljust(division_length, '-'))
result = 'Yes' if B_PER_index in input_predict_word_index or I_PER_index in input_predict_word_index else "No"
print("Does this sentence have any named entity of persons?", result)
if result == 'Yes':
    print()
    for word, ner_index in zip(input_sentence_list, input_predict_word_index):
        if ner_index == B_PER_index or ner_index == I_PER_index:
            print(word[0], '<PER>')
        else:
            print(word[0])
print("-" * division_length)
print()
print()

print(input_predict_word_index)
print(ner_to_index_dic)

Input your sentence: aa




Instructions for updating:
Please use `rate` instead of `keep_prob`. Rate should be set to `rate = 1 - keep_prob`.
Instructions for updating:
Use tf.where in 2.0, which has the same broadcast rule as np.where


result  ------------------------------------------------------------------------
Does this sentence have any named entity of persons? No
--------------------------------------------------------------------------------


[2]
{'PAD': 0, 'B-ORG': 1, 'O': 2, 'B-MISC': 3, 'B-PER': 4, 'I-PER': 5, 'B-LOC': 6, 'I-ORG': 7, 'I-MISC': 8, 'I-LOC': 9}
